# Geochemical Data - Importing, Processing and 'Munging'

In this notebook we'll go through some of the tasks which `pyrolite` can help make a bit easier with respect to getting your data analysis ready.


## Importing Data

`pyrolite` is largely based around `pandas`, and as such you're typically working with Pandas dataframes. Pandas can work with [a variety of file types](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html), some more performant than others, but is also happy to digest humble CSV and Excel files (with the functions `pandas.read_csv` and `pandas.read_excel`, respectively). Some of these functions are able to directly connect to remote files (e.g. CSV) or directly to database connections. You'll see one or two examples of fetching a remote CSV file directly below and in other notebooks.

## Cleaning Up Column Names

One of the challenges of working with larger datasets is being able to quickly find the right data when you need it. `pyrolite` provides some functions for this, but for the time being is dependent on being able to recognise compositional columns by looking for elements, oxide names and isotopes (without unit annotations, delimiters and other markup). Here we show some of the steps which might be required to get your dataframe in a standardised format (using examples [from GEOROC](http://georoc.mpch-mainz.gwdg.de/georoc/Entry.html)). Notably, this can be the most difficut step of any analysis workflow, so being able to do this in a way which is repeatable might save you a decent amount of time if you have to do it again in the future!

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('http://georoc.mpch-mainz.gwdg.de/georoc/Csv_Downloads/Continental_Flood_Basalts_comp/CENTRAL_ATLANTIC_MAGMATIC_PROVINCE_-_CAMP.csv',
                 encoding='cp1252',
                 skip_blank_lines=False) # get some data from GEOROC directly
df = df.loc[:np.argmax(df.iloc[:, 0].isnull())-1] # omit the abbreviations and references after the blank line in this file

Quickly looking at this dataframe, we can see that all the column names are capitalised, `_` is used as a delimiter for isotopes, and units are given for the geochemical parameters in the format `(UNIT)`. We also have a unique index, and ean extra redundant column on the right hand side of our table (which we can drop):

In [2]:
df.head(2)

,CITATIONS,TECTONIC SETTING,LOCATION,LOCATION COMMENT,LATITUDE MIN,LATITUDE MAX,LONGITUDE MIN,LONGITUDE MAX,LAND OR SEA,ELEVATION MIN,...,RE187_OS188,HF176_HF177,HE3_HE4,HE3_HE4(R/R(A)),HE4_HE3,HE4_HE3(R/R(A)),K40_AR40,AR40_K40,UNIQUE_ID,Unnamed: 171
0,[20054],CONTINENTAL FLOOD BASALT,CENTRAL ATLANTIC MAGMATIC PROVINCE - CAMP / SI...,"NEAR LEONFORTE VILLAGE, ALONG THE SOUTHERN SLO...",37.6403,37.6403,134.3278,134.3278,SAE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1015617,NaN
1,[20054],CONTINENTAL FLOOD BASALT,CENTRAL ATLANTIC MAGMATIC PROVINCE - CAMP / SI...,"NEAR LEONFORTE VILLAGE, ALONG THE SOUTHERN SLO...",37.6403,37.6403,134.3278,134.3278,SAE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1015618,NaN


In [3]:
df.drop(columns=[c for c in df.columns if 'Unnamed' in c], inplace=True) # drop our redundant column

We can alter the names of columns which `pyrolite` will recognise so we can use some of its more automated methods. Specifically, pyrolite expects element and oxide names to have 'title case' names (e.g. `Si`, `SiO2`, not `MG`, `MGO`). Similarly, for isotope ratios, it expects something along the lines of `87Sr/86Sr`. Here I've written a function which will attenpt to find relevant element and oxide names among the capitalised versions we find here.

In [4]:
from pyrolite.geochem.ind import __common_elements__, __common_oxides__ # indexes of elements and oxides which we'll check against

def rename_columns(df):
    """
    Rename the columns which pyrolite can access so we can use the indexing and transformation functions.
    
    Parameters
    ----------
    df : pandas.DataFrame
        Dataframe with columns you'd like to rename.
    
    Returns
    -------
    pandas.DataFrame
        Dataframe with columns renamed.
    """
    _elements, _oxides = {e.upper(): e for e in __common_elements__}, {o.upper(): o for o in __common_oxides__} # these will serve as lookup tables for our capitalised components
    
    element_columns = {c: _elements[c[:c.find("(PPM")]] for c in df.columns if c[:c.find("(PPM")] in _elements} # all of the elemental values are in ppm here
    oxide_columns = {c: _oxides[c[:c.find("(WT%")]] for c in df.columns if c[:c.find("(WT%")] in _oxides} # this omits some gas measurements, but gets the major oxides
    isotope_columns = {c: '/'.join([c.title() for c in c.split('_')]) for c in df.pyrochem.list_isotope_ratios if len(c.split('_'))==2}
    
    return df.rename(columns = {**element_columns, **oxide_columns, **isotope_columns})
    

df = rename_columns(df)

In [5]:
df.head(2)

,CITATIONS,TECTONIC SETTING,LOCATION,LOCATION COMMENT,LATITUDE MIN,LATITUDE MAX,LONGITUDE MIN,LONGITUDE MAX,LAND OR SEA,ELEVATION MIN,...,Re187/Os186,Re187/Os188,Hf176/Hf177,He3/He4,He3/He4(R/R(A)),He4/He3,He4/He3(R/R(A)),K40/Ar40,Ar40/K40,UNIQUE_ID
0,[20054],CONTINENTAL FLOOD BASALT,CENTRAL ATLANTIC MAGMATIC PROVINCE - CAMP / SI...,"NEAR LEONFORTE VILLAGE, ALONG THE SOUTHERN SLO...",37.6403,37.6403,134.3278,134.3278,SAE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1015617
1,[20054],CONTINENTAL FLOOD BASALT,CENTRAL ATLANTIC MAGMATIC PROVINCE - CAMP / SI...,"NEAR LEONFORTE VILLAGE, ALONG THE SOUTHERN SLO...",37.6403,37.6403,134.3278,134.3278,SAE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1015618


Finally, it might be a good idea to use the Unique ID as the index for our dataframe:

In [6]:
df = df.set_index('UNIQUE_ID', drop=True)

You could package this up into a function to fetch one of these GEOROC files:

In [7]:
def fetch_GEOROC_csv(filepath):
    """
    Fetch a GEOROC csv from a local file or URL.
    
    Parameters
    ----------
    filepath : str | pathlib.Path
        Filepath to a GEOROC csv - can be used to directly fetch a URL thanks to `pandas.read_csv`.
    
    Returns
    -------
    pandas.DataFrame
        Dataframe formatted for `pyrolite`.
    """
    df = pd.read_csv(filepath, encoding='cp1252', skip_blank_lines=False) # get some data from GEOROC directly
    df = df.loc[:np.argmax(df.iloc[:, 0].isnull())-1] # omit the abbreviations and references after the blank line in this file
    df.drop(columns=[c for c in df.columns if 'Unnamed' in c], inplace=True) # drop our redundant column
    df = rename_columns(df)
    df = df.set_index('UNIQUE_ID', drop=True)
    return df

Now we can use this to fetch another CSV - this time from the Kermadec Arc. Note that the column names which `pyrolite` can work with are already converted to usable versions:

In [8]:
kermadec_df = fetch_GEOROC_csv('http://georoc.mpch-mainz.gwdg.de/georoc/Csv_Downloads/Convergent_Margins_comp/KERMADEC_ARC.csv')
kermadec_df.head(2)

,CITATIONS,TECTONIC SETTING,LOCATION,LOCATION COMMENT,LATITUDE MIN,LATITUDE MAX,LONGITUDE MIN,LONGITUDE MAX,LAND OR SEA,ELEVATION MIN,...,Os187/Os188,Re187/Os186,Re187/Os188,Hf176/Hf177,He3/He4,He3/He4(R/R(A)),He4/He3,He4/He3(R/R(A)),K40/Ar40,Ar40/K40
UNIQUE_ID,,,,,,,,,,,,,,,,,,,,,
10203-46325,[13460][9592],CONVERGENT MARGIN,KERMADEC ARC / KERMADEC ISLANDS / RAOUL / RAOUL,NaN,-29.25,-29.25,-177.87,-177.87,SAE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10203-46329,[13460][9592],CONVERGENT MARGIN,KERMADEC ARC / KERMADEC ISLANDS / RAOUL / RAOUL,NaN,-29.25,-29.25,-177.87,-177.87,SAE,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Selecting Subsets of your Data

The `pyrolite.pyrochem` API provides access to indexing and transformation functions. This allows easy subsetting of geochemical datasets which can otherwise be unweildly (expecially as the number of columns increases..). To provide a simple illustration we generate a synthetic dataset to work from, which contains an array of typical geochemical measures - oxide components, element components (here as ppm), element ratios and isotope ratios. While this size dataset is managable, some of the indexing tools pyrolite provides make it straightforward to pull out different parts of the dataset.

In [9]:
import numpy as np
from pyrolite.util.synthetic import normal_frame

df = normal_frame(columns=['CaO', 'MgO', 'SiO2', 'FeO','Na2O', 'Ni', 'Ti', 'La', 'Lu', 'Te']) * 100
df[['Ni', 'Ti', 'La', 'Lu', 'Te']] *= 10
df.pyrochem.add_ratio('Mg/Fe') # one way to add an element ratio to a dataframe!
df['Sr87/Sr86'] = 0.0700  / 0.0986 + np.random.randn(df.index.size) * 0.0001
df

,CaO,MgO,SiO2,FeO,Na2O,Ni,Ti,La,Lu,Te,Mg/Fe,Sr87/Sr86
0,7.179336,4.871556,7.598356,13.081688,5.179388,65.717354,34.632595,84.669443,304.377858,131.499507,0.288905,0.709920
1,7.318218,5.225071,7.663196,13.407155,5.334561,66.690819,34.136224,83.176119,295.544270,130.970554,0.302348,0.709792
2,7.246025,5.110169,7.776407,13.455016,5.193477,65.904254,34.777754,83.863067,300.914243,126.729752,0.294648,0.709976
3,7.318678,5.139519,7.606591,13.720662,5.271950,65.460281,34.351101,84.383120,291.239217,133.992275,0.290602,0.709942
4,7.311988,4.950609,8.020503,14.455369,5.547092,63.885056,35.035889,81.711895,289.772602,126.738948,0.265694,0.709973
5,6.993433,4.896143,7.584241,13.532575,5.245288,64.385798,34.619532,83.522711,309.597569,125.357584,0.280689,0.709910
6,7.471275,5.279261,7.882833,13.389229,5.489877,65.388570,33.168378,86.718851,284.188392,135.411054,0.305893,0.709946
7,7.030862,5.369727,7.608409,13.412674,5.239864,66.446655,34.736082,82.763380,298.985298,130.453223,0.310591,0.709926
8,7.167209,5.026813,7.926034,13.640663,5.445549,65.078762,34.697551,81.228793,294.195852,132.736360,0.285897,0.709923
9,7.181761,4.965163,7.815084,12.939586,5.150317,63.882097,35.625079,79.983862,317.351340,122.638521,0.297690,0.710097


In [10]:
df.pyrochem.oxides

,CaO,MgO,SiO2,FeO,Na2O
0,7.179336,4.871556,7.598356,13.081688,5.179388
1,7.318218,5.225071,7.663196,13.407155,5.334561
2,7.246025,5.110169,7.776407,13.455016,5.193477
3,7.318678,5.139519,7.606591,13.720662,5.271950
4,7.311988,4.950609,8.020503,14.455369,5.547092
5,6.993433,4.896143,7.584241,13.532575,5.245288
6,7.471275,5.279261,7.882833,13.389229,5.489877
7,7.030862,5.369727,7.608409,13.412674,5.239864
8,7.167209,5.026813,7.926034,13.640663,5.445549
9,7.181761,4.965163,7.815084,12.939586,5.150317


In [11]:
df.pyrochem.elements

,Ni,Ti,La,Lu,Te
0,65.717354,34.632595,84.669443,304.377858,131.499507
1,66.690819,34.136224,83.176119,295.544270,130.970554
2,65.904254,34.777754,83.863067,300.914243,126.729752
3,65.460281,34.351101,84.383120,291.239217,133.992275
4,63.885056,35.035889,81.711895,289.772602,126.738948
5,64.385798,34.619532,83.522711,309.597569,125.357584
6,65.388570,33.168378,86.718851,284.188392,135.411054
7,66.446655,34.736082,82.763380,298.985298,130.453223
8,65.078762,34.697551,81.228793,294.195852,132.736360
9,63.882097,35.625079,79.983862,317.351340,122.638521


In [12]:
df.pyrochem.REE

,La,Lu
0,84.669443,304.377858
1,83.176119,295.544270
2,83.863067,300.914243
3,84.383120,291.239217
4,81.711895,289.772602
5,83.522711,309.597569
6,86.718851,284.188392
7,82.763380,298.985298
8,81.228793,294.195852
9,79.983862,317.351340


In [13]:
df.pyrochem.compositional

,CaO,MgO,SiO2,FeO,Na2O,Ni,Ti,La,Lu,Te
0,7.179336,4.871556,7.598356,13.081688,5.179388,65.717354,34.632595,84.669443,304.377858,131.499507
1,7.318218,5.225071,7.663196,13.407155,5.334561,66.690819,34.136224,83.176119,295.544270,130.970554
2,7.246025,5.110169,7.776407,13.455016,5.193477,65.904254,34.777754,83.863067,300.914243,126.729752
3,7.318678,5.139519,7.606591,13.720662,5.271950,65.460281,34.351101,84.383120,291.239217,133.992275
4,7.311988,4.950609,8.020503,14.455369,5.547092,63.885056,35.035889,81.711895,289.772602,126.738948
5,6.993433,4.896143,7.584241,13.532575,5.245288,64.385798,34.619532,83.522711,309.597569,125.357584
6,7.471275,5.279261,7.882833,13.389229,5.489877,65.388570,33.168378,86.718851,284.188392,135.411054
7,7.030862,5.369727,7.608409,13.412674,5.239864,66.446655,34.736082,82.763380,298.985298,130.453223
8,7.167209,5.026813,7.926034,13.640663,5.445549,65.078762,34.697551,81.228793,294.195852,132.736360
9,7.181761,4.965163,7.815084,12.939586,5.150317,63.882097,35.625079,79.983862,317.351340,122.638521


In [14]:
df.pyrochem.isotope_ratios

,Sr87/Sr86
0,0.709920
1,0.709792
2,0.709976
3,0.709942
4,0.709973
5,0.709910
6,0.709946
7,0.709926
8,0.709923
9,0.710097


Notably, these dataframe accessors can also be used to re-assign values back to the dataframe. Here we transform element components to wt% equivalents by dividing by 10000, and note that the change has been incorporated into our dataframe:

In [15]:
df.pyrochem.elements /= 10000
df.pyrochem.elements

,Ni,Ti,La,Lu,Te
0,0.006572,0.003463,0.008467,0.030438,0.013150
1,0.006669,0.003414,0.008318,0.029554,0.013097
2,0.006590,0.003478,0.008386,0.030091,0.012673
3,0.006546,0.003435,0.008438,0.029124,0.013399
4,0.006389,0.003504,0.008171,0.028977,0.012674
5,0.006439,0.003462,0.008352,0.030960,0.012536
6,0.006539,0.003317,0.008672,0.028419,0.013541
7,0.006645,0.003474,0.008276,0.029899,0.013045
8,0.006508,0.003470,0.008123,0.029420,0.013274
9,0.006388,0.003563,0.007998,0.031735,0.012264


If you're just after a list of the relevant column names, there are respective functions for that too:

In [16]:
df.pyrochem.list_oxides

['CaO', 'MgO', 'SiO2', 'FeO', 'Na2O']

## Unit Scales

While you can transform elements and oxide abundnace units easily when you remember the relative scales, `pyrolite` provides some functions such that you don't have to rely on your memory. Here we create a copy of the dataframe and within it revert the change we made above - so these should be the orignal ppm values. This method provides an easy way to explicitly declare your intention when changing units - and makes sure the relative scales are correct!

In [17]:
df.pyrochem.elements.pyrochem.scale('wt%', 'ppm') # wt% to ppm

,Ni,Ti,La,Lu,Te
0,65.717354,34.632595,84.669443,304.377858,131.499507
1,66.690819,34.136224,83.176119,295.544270,130.970554
2,65.904254,34.777754,83.863067,300.914243,126.729752
3,65.460281,34.351101,84.383120,291.239217,133.992275
4,63.885056,35.035889,81.711895,289.772602,126.738948
5,64.385798,34.619532,83.522711,309.597569,125.357584
6,65.388570,33.168378,86.718851,284.188392,135.411054
7,66.446655,34.736082,82.763380,298.985298,130.453223
8,65.078762,34.697551,81.228793,294.195852,132.736360
9,63.882097,35.625079,79.983862,317.351340,122.638521


## Converting Chemical Components 

`pyrolite` provides some straightfoward methods to calcuate element-oxide conversions (e.g. to transform Ti abundance to TiO2 abudnance), assuming that the system is open to oxygen (i.e. in this case the extra oxygen will be added to the composition). This interface also allows the user to quickly add ratios and specify redox pairs at the same time. For example, we can transform a copy of our dataframe to include extra ratios and change some of our oxide components to elements:

In [18]:
df.pyrochem.convert_chemistry(
    to=["MgO", "SiO2", "FeO", "Ca", "Te", "Na", "Na/Te", "MgO/SiO2"]
)

,Mg/Fe,Sr87/Sr86,MgO,SiO2,FeO,Ca,Te,Na,Na/Te,MgO/SiO2
0,0.288905,0.709920,4.871556,7.598356,13.081688,5.131005,0.013150,3.842368,292.196358,0.641133
1,0.302348,0.709792,5.225071,7.663196,13.407155,5.230263,0.013097,3.957484,302.165937,0.681840
2,0.294648,0.709976,5.110169,7.776407,13.455016,5.178667,0.012673,3.852819,304.018545,0.657138
3,0.290602,0.709942,5.139519,7.606591,13.720662,5.230592,0.013399,3.911036,291.885157,0.675667
4,0.265694,0.709973,4.950609,8.020503,14.455369,5.225811,0.012674,4.115152,324.695098,0.617244
5,0.280689,0.709910,4.896143,7.584241,13.532575,4.998142,0.012536,3.891256,310.412478,0.645568
6,0.305893,0.709946,5.279261,7.882833,13.389229,5.339651,0.013541,4.072706,300.766175,0.669716
7,0.310591,0.709926,5.369727,7.608409,13.412674,5.024892,0.013045,3.887232,297.979022,0.705762
8,0.285897,0.709923,5.026813,7.926034,13.640663,5.122338,0.013274,4.039822,304.349275,0.634215
9,0.297690,0.710097,4.965163,7.815084,12.939586,5.132738,0.012264,3.820801,311.549844,0.635331


In a similar way, we can also specify the molar speciation for redox species (so far just iron; others could be incorporated if they'll be useful). Here we adjust the total iron within our compositions (currently specified as FeO) to have a $Fe^{2+}/Fe^{3+}$ ratio of 9:1 (roughly what you might expect from a normal mantle-derived magma):

In [19]:
df.pyrochem.convert_chemistry(to=[{"FeO": 0.9, "Fe2O3": 0.1}])

,Mg/Fe,Sr87/Sr86,FeO,Fe2O3
0,0.288905,0.709920,11.773519,1.453830
1,0.302348,0.709792,12.066439,1.490001
2,0.294648,0.709976,12.109514,1.495320
3,0.290602,0.709942,12.348596,1.524842
4,0.265694,0.709973,13.009832,1.606494
5,0.280689,0.709910,12.179318,1.503939
6,0.305893,0.709946,12.050306,1.488009
7,0.310591,0.709926,12.071406,1.490614
8,0.285897,0.709923,12.276597,1.515952
9,0.297690,0.710097,11.645627,1.438038
